# Maticové metody analýzy lineárních obvodů


In [1]:
import sympy as sp
import numpy as np
from numpy import pi
from matplotlib import pyplot as plt
from copy import deepcopy
from IPython.display import display, Math

def tex(*arg):
    tmp = []
    for a in arg:
        if type(a) is str:
            
            format_hat = False
            format_bolt = False
            
            if a == '*':
                a = '\cdot'
            
            if a.startswith('__'):
                format_bolt = True
                a = a[2:]
            
            if a.startswith('^'):
                format_hat = True
                a = a[1:]
            
            if format_bolt:
                a = f'\\boldsymbol{{{a}}}'
            if format_hat:
                a = f'\\hat{{{a}}}'

        tmp.append(sp.latex(a))    
    display(Math(' '.join(tmp)))

## Incidenční matice smyček a větví

### Smyček

* $d_{ik}=1$ když se orientace i-té smyčky shoduje s orientací k-té větve
* $d_{ik}=-1$ když je orientace i-té smyčky opačná s orientací k-té větve
* $d_{ik}=0$ když k-tá větev není součástí i-té smyčky
* Incidenční matice se označuje __D__

### Větví

* $m_{ik}=1$ když z i-tého uzlu k-tá větev vystupuje
* $m_{ik}=-1$ když do i-tého uzlu k-tá větev vstupuje
* $m_{ik}=0$ když i-tý uzel s k-tou větví nesouvisí
* Incidenční matice se označuje __M__

Jsou-li větve v incidenčních maticích __D__ a __M__ uspořádány stejně, potom platí ortogonální vztahy:
$$ \boldsymbol{M^TD = D^TM = 0} $$

* Incidenční matice umožňují algoritmizování maticových metod pro řešení na počítači
* Veškerá větvová napětí lze vyjádřit jako rozdíly uzlových napětí, tedy maticovým zápisem
$$ \boldsymbol{ U_{\text{VĚTEV}} = M^T U_{\text{UZEL}} } $$

## MSPM - Metoda smyčkových proudů (Loop Current Method)
* motivací je __zmenšení__ počtu __řešených rovnic__, __zavedeme__ (zvolíme) __fiktivní__ smyčkové __proudy__ $I_S$ (nové neznámé) mezi nimi a skutečnými proudy (větvemi) I_V bude platit: $$I_V = D^TI_S$$
* kde __D__ je incidenční matice smyček a větví
* volíme __vějířovitý tvar__ úplného __stromu__ soustavy, potom jsou jednotlivé smyčky přehledně rozprostřeny a __dotýkají se minimálním počtem větví__

### Postup řešení u metody smyčkových proudů
* __Nelze řešit obvody s nezávislými zdroji proudu__
* Budící zdroje proudu přepočítáme na ekvivalentní zdroje napětí
* __Sestavíme__ s __nezávislých rovnic__ podle __druhého Kirchhoffova zákona__ (lineární nehomogenní soustava rovnic)
* __Řešení__ získáme __inverzí impedanční matice__

$$ U = ZI $$
$$ I = Z^{-1}U $$

Kde:
* __U__ je vektor známých uzlových napětí
* __Z__ je impedanční matice soustavy
* __I__ je vektor neznámých smyčkových proudů

$$ I_{S1} = \frac{\Delta_1}{\Delta}, I_{S2} = \frac{\Delta_2}{\Delta}, ... $$

## Algoritmizace metody smyčkových proudů

 * Vychází s využití __incidenčních matic__ a __impedanční matice větví__ $Z_V$ což je čtvercová diagonální matice s __impedancemi větví__ v __hlavní diagonále__ 
(jinak jsou prvky nulové)

$$ Z_S = DZ_VD^T $$

* Matice $Z_V$ je součástí impedančního popisu jednotlivých větví obvodu v globálním tvaru

$$ U_V = Z_VI_V $$

* Kde: $U_V$ a $I_V$ jsou vektory napětí a proudů všech větví

* U elektronických __obvodů obsahující regulární dvojbrany__ je výše uvedený __postup nezbytný__
* U __obvodů obsahujících pouze dvojpóly__ lze __snáze získat impedanční matici__ postupem:
    1. prvky v hlavní diagonále jsou součtem impedancí všech dvojpólů zapojených ve větvích příslušné smyčky
    2. prvky mimo hlavní diagonálu jsou vždy rovny dvojpólu zapojenému ve větvi společné oběma smyčkám, přičemž znaménko je kladné při shodném směru smyčkových proudů

In [2]:
s = sp.Symbol('s')

R1 = sp.Symbol('R_1')
R2 = sp.Symbol('R_2')
R3 = sp.Symbol('R_3')
R4 = sp.Symbol('R_4')
R5 = sp.Symbol('R_5')
L1 = sp.Symbol('L_1')
L2 = sp.Symbol('L_2')
C1 = sp.Symbol('C_1')
C2 = sp.Symbol('C_2')

Z_L1 = s*L1
Z_L2 = s*L2
Z_C1 = 1/(s*C1)
Z_C2 = 1/(s*C2)

Iv1 = sp.Symbol('I_{V1}')
Iv2 = sp.Symbol('I_{V2}')
Iv3 = sp.Symbol('I_{V3}')
Iv4 = sp.Symbol('I_{V4}')
Iv5 = sp.Symbol('I_{V5}')
Iv6 = sp.Symbol('I_{V6}')
Iv7 = sp.Symbol('I_{V7}')
Iv = sp.Matrix([Iv1,Iv2,Iv3,Iv4,Iv5,Iv6,Iv7])

Is1 = sp.Symbol('I_{S1}')
Is2 = sp.Symbol('I_{S2}')
Is3 = sp.Symbol('I_{S3}')
Is4 = sp.Symbol('I_{S4}')
Is = sp.Matrix([Is1,Is2,Is3,Is4])

U1 = sp.Symbol('U_1')
Us = sp.Matrix([U1,0,0,0])


### Ukázkový příklad:
![](img/sch_msp.png)

In [3]:
D = sp.Matrix([
    [1,1,1,0,0,0,0],
    [0,0,0,0,-1,1,0],
    [0,0,-1,1,1,0,0],
    [0,-1,0,-1,0,0,1]
])
tex('__D', '=', D)

<IPython.core.display.Math object>

In [4]:
tex('__D', '*', '__I_V', '=', '__I_S')
tex(D, '*', Iv, '=', D*Iv, '=', Is)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [5]:
tex('__D^T', '*', '__I_S', '=', '__I_V')
tex(D.T, '*', Is, '=', D.T * Is, '=', Iv)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [6]:
Zv = sp.Matrix([
    [R1,0,0,0,0,0,0],
    [0,Z_C1,0,0,0,0,0],
    [0,0,R2,0,0,0,0],
    [0,0,0,Z_L1+R4,0,0,0],
    [0,0,0,0,Z_C2,0,0],
    [0,0,0,0,0,R5+Z_L2,0],
    [0,0,0,0,0,0,R3]
])
tex('Z_V=', Zv)

<IPython.core.display.Math object>

In [7]:
Zs = D * Zv * D.T
tex('Z_S = D Z_V D^T=', Zs)
tex(D,'*',Zv,'=',D * Zv)
tex(D*Zv,'*',D.T,'=',Zs)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [8]:
tex(Us,'=',Zs,'*',Is)

<IPython.core.display.Math object>

In [9]:
M1 = deepcopy(Zs)
M1[:, 0] = Us
tex(M1)

M2 = deepcopy(Zs)
M2[:, 1] = Us
tex(M2)

M3 = deepcopy(Zs)
M3[:, 2] = Us
tex(M3)

M4 = deepcopy(Zs)
M4[:, 3] = Us
tex(M4)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [10]:
det = Zs.det()
det1 = M1.det()
det2 = M2.det()
det3 = M3.det()
det4 = M4.det()

tex('\Delta=', det)
tex('\Delta_1=', det1)
tex('\Delta_2=', det2)
tex('\Delta_3=', det3)
tex('\Delta_4=', det4)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [11]:
Is1 = det1 / det
Is2 = det2 / det
Is3 = det3 / det
Is4 = det4 / det
tex('I_{S1}=', Is1)
tex('I_{S2}=', Is2)
tex('I_{S3}=', Is3)
tex('I_{S4}=', Is4)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>